In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os,shutil
from os.path import isfile, join

In [2]:
#video capturing

cap = cv2.VideoCapture('ovideo.mp4')


In [ ]:

count=0
# plt.figure()
# deleting files from answer folder which will hold frames for output video
folder = 'answer/'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
# cleaning intermidiate folder which save frame of input video
folder = 'inter/'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)


#reading frames from input video and saving it in inter folder
count = 10
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(range(length))
for count in range(length):
    count += 1
    ret, frame = cap.read()
    cv2.imwrite("inter/frame%d.jpg" % count, frame)
    print('Read a new frame: ', count, length)

cap.release()

#generating depth images from frames which are present in inter folder
t=1
while t+30 <= count:
    print('loading images...',t)
    imgL = cv2.imread('inter/frame'+str(t)+'.jpg')  # downscale images for faster processing
    imgR = cv2.imread('inter/frame'+str(t+30)+'.jpg')
    t+=1
    disparityrange = 160
    zero = np.zeros((imgL.shape[0], disparityrange, 3))
    imgL = np.hstack((zero, imgL))
    imgR = np.hstack((zero, imgR))
    imgL = imgL.astype(np.uint8)
    imgR = imgR.astype(np.uint8)
    window_size = 3  # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
    left_matcher = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=disparityrange,  # max_disp has to be dividable by 16 f. E. HH 192, 256
        blockSize=5,
        P1=8 * 3 * window_size ** 2,
        # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
        P2=32 * 3 * window_size ** 2,
        disp12MaxDiff=-1,
        uniquenessRatio=15,
        speckleWindowSize=0,
        speckleRange=2,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)
    lmbda = 80000
    sigma = 1.2
    visual_multiplier = 1

    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)
    displ = left_matcher.compute(imgL, imgR)  # .astype(np.float32)/16
    dispr = right_matcher.compute(imgR, imgL)  # .astype(np.float32)/16

    displ = np.int16(displ)
    dispr = np.int16(dispr)

    filteredImg = wls_filter.filter(displ, imgL, None, dispr)  # important to put "imgL" here!!!
    filteredImg = cv2.normalize(src=filteredImg, dst=filteredImg, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX);
    filteredImg = np.uint8(filteredImg)
    filteredImg = np.delete(filteredImg, np.s_[0:disparityrange], axis=1)
    path = 'answer/'
    # saving depth images in answer folder
    path = path + str(t) + '.jpg'
    cv2.imwrite(path, filteredImg)
    print(str(t) + 'done')

    #video generation from answwer folder
    pathIn= 'answer/'
    pathOut = 'video.avi'
    fps = 25.0

    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

    # for sorting the file names properly
    # files.sort()

    print(files)
    size=(0,0)
    for i in range(len(files)):
        filename = pathIn +str(i+2)+'.jpg'
        print(filename)
        # reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        print(filename)
        # inserting the frames into an image array
        frame_array.append(img)

    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
